In [ ]:
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.animation import FuncAnimation, FFMpegWriter
from IPython.display import HTML

sp.init_printing(use_latex='mathjax')

In [ ]:
# import matplotlib
# matplotlib.rcParams['animation.embed_limit'] = matplotlib.rcParams['animation.embed_limit'] * 10
# matplotlib.use('WebAgg')

In [ ]:
# Stałe zdobyte z artukułu
N_AVG = 320
SIGMA = 2.5

def plot_rydberg_packet(n_avg=N_AVG, sigma=SIGMA, time_fraction=0.0, resolution=500):
    """
    Generates and plots a Circular-Orbit Rydberg Wave Packet.
    
    Parameters:
    - n_avg: The central principal quantum number (e.g., 320).
    - sigma: The spread of the Gaussian distribution (e.g., 2.5).
    - time_fraction: Time evolved as a fraction of the Kepler period (0.0 to 1.0).
    - resolution: Grid resolution (higher = smoother but slower).
    """
    # poziomy energetyczne które będziemy uwzględniać na naszym wykresie
    # ('3 * standardowe odczylenie zgarnia ~99.7% pakietów energetycznych' ~ Gemini)
    n_range = np.arange(int(n_avg - 4*sigma), int(n_avg + 4*sigma) + 1)

    r_min = n_avg**2 - (40 * n_avg)
    r_max = n_avg**2 + (40 * n_avg)

    r_vals = np.linspace(r_min, r_max, resolution)
    
    phi_vals = np.linspace(0, 2*np.pi, resolution)
    
    R, PHI = np.meshgrid(r_vals, phi_vals)
    
    Psi_total = np.zeros_like(R, dtype=np.complex128)

    # Kepler Period T = 2 * pi * n^3
    T_kepler = 2 * np.pi * (n_avg**3)

    # Przybliżony czas na powrót do stanu początkowego
    T_rev = (n_avg / 3) * T_kepler

    current_time = time_fraction * T_rev

    # We use Logarithms for the radial part to prevent overflow (r^300 is huge!)
    for n in n_range:
        weight = np.exp(-((n - n_avg)**2) / (4 * sigma**2))
        
        # Stirling's approximation for (2n)!
        # Stała normalizująca tylko zlogarytmowana, bo inaczej by wywalało overflowa
        log_norm = -0.5 * ((2*n)*np.log(2*n) - (2*n) + 0.5*np.log(2*np.pi*2*n)) + (n + 0.5)*np.log(2/n)
        
        # Log of the radial part
        # Stała powiązana z prawdopodobieństwami pojawienia się elkronu w stronę R, też zlogarytmowana
        log_radial_part = (n - 1) * np.log(R + 1e-10) - (R / n)
        
        # COMBINED LOG EXPONENT 
        # Dostajemy całą stałą normalizującą przy czym dodajemy zamiast mnożyć bo są zlogarytmowane
        # Normalnie byśmy je mnożyli
        combined_log = log_radial_part + log_norm
        
        # SHIFTING: Substract the max value of the exponent to prevent overflow
        # Wracamy z naszą stałą do odpowiedniej wartości rzeczywistej aby już nie była w logarytmach
        shift = np.max(combined_log)
        radial_amp = np.exp(combined_log - shift)
        
        # Angular and Time Phases
        angular_phase = np.exp(1j * (n - 1) * PHI)
        time_phase = np.exp(1j * current_time / (2 * n**2))
        
        Psi_total += weight * radial_amp * angular_phase * time_phase

    Prob_Density = np.abs(Psi_total)**2
    
    Prob_Density /= np.max(Prob_Density)

    # Convert Polar (R, PHI) to Cartesian (X, Y) for the 3D plot surface
    X = R * np.sin(PHI)
    Y = R * np.cos(PHI)

    return X, Y, Prob_Density


In [ ]:
TIME_FRACTION = 0.993

fig, ax = plt.subplots(subplot_kw={"projection": "3d"}, figsize=(10, 8))

# Plot surface
surf = ax.plot_surface(*plot_rydberg_packet(time_fraction=TIME_FRACTION), 
                       cmap=cm.CMRmap, linewidth=0,
                        antialiased=True, rcount=500, ccount=300)

ax.set_title(f"Rydberg Wave Packet (n={N_AVG})\nTime = {TIME_FRACTION:.2f} Kepler Periods")
ax.set_xlabel("x (atomic units)")
ax.set_ylabel("y (atomic units)")
ax.set_zlabel("Probability Density")

plt.show()

In [ ]:
fig, ax = plt.subplots(subplot_kw={"projection": "3d"}, figsize=(10, 8))

# Define the frames (e.g., 50 steps from t=0 to t=1 T_rev)
frames = np.linspace(0, 0.9883, 6) 

def update(frame_fraction):
    ax.clear() # Clear the previous mountain
    
    # Get new data for this specific time
    X, Y, Z = plot_rydberg_packet(time_fraction=frame_fraction, resolution=300)
    
    # Redraw the surface
    surf = ax.plot_surface(X, Y, Z, cmap=cm.CMRmap, 
                           linewidth=0, antialiased=True, 
                           rcount=500, ccount=300)
    
    # Update timestamp and labels
    # Note: If your function uses T_rev, label it as T_rev
    ax.set_title(f"Rydberg Wave Packet (n={N_AVG})\nTime = {frame_fraction:.2f} $T_{{rev}}$")
    ax.set_zlim(0, np.max(Z) * 1.1) # Keep the Z-axis stable
    return surf,

# Create the animation
ani = FuncAnimation(fig, update, frames=frames, interval=100)

# Display in Jupyter
plt.close() # Prevents a static plot from showing up below
HTML(ani.to_jshtml())

In [ ]:
n, n_avg, sigma = sp.symbols(r'n \bar{n} sigma')
phi, r, t = sp.symbols('phi r t')
c_1 = sp.symbols('C_1')

In [ ]:
weight_expr = sp.exp(-(n - n_avg)**2 / (4*sigma**2))
weight_expr

In [ ]:
angular_phase_expr = sp.exp(sp.I * (n - 1) * phi)
angular_phase_expr

In [ ]:
time_phase_expr = sp.exp(sp.I * t / (2 * n**2))
time_phase_expr

In [ ]:
Psi_total_expr = weight_expr * c_1 * angular_phase_expr * time_phase_expr
Psi_total_expr

In [ ]:
log_norm_expr = -0.5 * ((2*n)*sp.log(2*n) - (2*n) + 0.5*sp.log(2*sp.pi*2*n)) + (n + 0.5)*sp.log(2/n)

log_radial_part_expr = (n - 1) * sp.log(r + 1e-10) - (r / n)

# COMBINED LOG EXPONENT 
# Dostajemy całą stałą normalizującą przy czym dodajemy zamiast mnożyć bo są zlogarytmowane
# Normalnie byśmy je mnożyli
combined_log_expr = log_radial_part_expr + log_norm_expr
combined_log_expr

In [ ]:
# SHIFTING: Substract the max value of the exponent to prevent overflow
# Wracamy z naszą stałą do odpowiedniej wartości rzeczywistej aby już nie była w logarytmach
# Częśc którą się korzysta w obliczeniach bo combined_log to tablica (dla rożnych R)
# C_max = shift = np.max(combined_log)
# radial_amp_expr = sp.exp(combined_log_expr - c_max)
# radial_amp_expr

In [ ]:
# Stałe z artykułu
N_AVG = 320
SIGMA = 2.5
RESOLUTION = 300

Psi_total_lambda = sp.lambdify((r, phi, t, n, c_1), Psi_total_expr.subs({sigma : SIGMA, n_avg : N_AVG}), 'numpy')
combined_log_lambda = sp.lambdify((r, n), combined_log_expr, 'numpy')

# poziomy energetyczne które będziemy uwzględniać na naszym wykresie
# ('3 * standardowe odczylenie zgarnia ~99.7% pakietów energetycznych' ~ Gemini)
n_range = np.arange(int(N_AVG - 4*SIGMA), int(N_AVG + 4*SIGMA) + 1)

r_min = N_AVG**2 - (40 * N_AVG)
r_max = N_AVG**2 + (40 * N_AVG)

r_vals = np.linspace(r_min, r_max, RESOLUTION)

phi_vals = np.linspace(0, 2*np.pi, RESOLUTION)

R, PHI = np.meshgrid(r_vals, phi_vals)

# Kepler Period T = 2 * pi * n^3
T_kepler = 2 * np.pi * (N_AVG**3)
# Przybliżony czas na powrót do stanu początkowego
T_rev = (N_AVG / 3) * T_kepler

def update(frame_fraction):
    ax.clear() # Clear the previous mountain
    
    Psi_total = np.zeros_like(R, dtype=np.complex128)

    X = R * np.sin(PHI)
    Y = R * np.cos(PHI)

    curr_t = frame_fraction*T_rev
    
    for n in n_range:
        c_1 = combined_log_lambda(R, n)
        c_max = np.max(c_1)

        Psi_total += Psi_total_lambda(R, PHI, curr_t, n, np.exp(c_1 - c_max))
    
    Prob_Density = np.abs(Psi_total)**2

    Z = Prob_Density/np.max(Prob_Density)
    
    # Redraw the surface
    surf = ax.plot_surface(X, Y, Z, cmap='CMRmap', 
                           linewidth=0, antialiased=True, 
                           rcount=500, ccount=300)
    
    # Update timestamp and labels
    # Note: If your function uses T_rev, label it as T_rev
    ax.set_title(f"Rydberg Wave Packet (n={N_AVG})\nTime = {frame_fraction:.3f} $T_{{rev}}$")
    ax.set_zlim(0, np.max(Z) * 1.1) # Keep the Z-axis stable
    return surf,

fig, ax = plt.subplots(subplot_kw={"projection": "3d"}, figsize=(10, 8))

# frames = np.linspace(0, 0.9883, 1600*3)
frames = np.linspace(0, 0.1, 267)

ani = FuncAnimation(fig, update, frames=frames, interval=50/3)

plt.close()
HTML(ani.to_jshtml())

In [ ]:
ani.save("rydberg_packet.gif", writer='pillow', fps=60)